In [105]:
import pandas as pd
import numpy as np
import os
import re

## Create and Preprocess DataSet

### Creating DataSet
Setting constants variables

In [128]:
csvs = os.listdir(DATA_PATH)
csv_regex = re.compile('*\.csv')
print(csv_regex.match("123.csv"))

error: nothing to repeat at position 0

In [118]:
csv_regex = re.compile('*.csv$')

DATA_PATH = './data/emtions-en-pt/'
CSV_FILES = [ file for file in os.listdir(DATA_PATH) if csv_regex.match(file)]

CSV_FILES

error: nothing to repeat at position 0

In [115]:
CSV_FILES

[]

Reading data

In [104]:
DATA_COLUMNS_TRANSLATOR = {'text':'en', 'texto':'pt'}
# create dataset
data = pd.DataFrame()

for csv_file in CSV_FILES:
    new_data = pd.read_csv(DATA_PATH + csv_file)
    data = pd.concat([data, new_data], ignore_index=True)
    
data.rename(columns=DATA_COLUMNS_TRANSLATOR, inplace=True)

IsADirectoryError: [Errno 21] Is a directory: './data/emtions-en-pt/dataset'

### Preprocessing DataSet

Analyzing data set

In [ ]:
data.head()

,en,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,pt
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,1,0,0,Aquele jogo doeu.
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,>sexualidade não deve ser uma categoria de agr...
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,1,"Você faz certo, se você não se importa, então ..."
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,0,0,0,0,0,0,0,0,0,"Cara, eu amo reddit."
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,1,"[NOME] não estava perto deles, ele estava pert..."


Analysing some 'example_very_unclea' samples

In [ ]:
data.loc[data['example_very_unclear']].sample(5)

,en,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,pt
116933,Yeah it’s weird bc in my family the basket cas...,eeozizq,botryoiidal,MurderedByWords,t3_aig0mn,t1_eenwb0i,1.548168e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,"Sim, é estranho porque na minha família o caso..."
151456,take my arrow. so fucking true,edcvb7d,it_is_mythic,CODZombies,t3_acyxp8,t3_acyxp8,1.546748e+09,22,True,0,...,0,0,0,0,0,0,0,0,0,pegue minha flecha. tão foda verdade
52130,Dissociation/ Derealization. Its a big symptom...,edj1205,basicallyaperson,SuicideWatch,t3_adoxwb,t3_adoxwb,1.546912e+09,61,True,0,...,0,0,0,0,0,0,0,0,0,Dissociação/ Desrealização. É um grande sintom...
157598,I just looked that up... Dear [NAME] why?,eedoxh7,MisterSquidInc,newzealand,t3_agvgna,t1_eeavqb5,1.547841e+09,61,True,0,...,0,0,0,0,0,0,0,0,0,Acabei de pesquisar... Querido [NOME] por quê?
58281,Options: 1. Hoax / joke 2. Crackpot 3. Wellmea...,ed6tpfc,bobledrew,ontario,t3_acboda,t1_ed6tkaf,1.546562e+09,61,True,0,...,0,0,0,0,0,0,0,0,0,Opções: 1. Hoax / piada 2. Crackpot 3. Bem-int...


Measuring data loss by removing all very unclear samples

In [ ]:
data['example_very_unclear'].value_counts()

example_very_unclear
False    207814
True       3411
Name: count, dtype: int64

Removing all very unclear samples 

In [ ]:
sf_data = data.loc[data['example_very_unclear'].map(lambda x: not x)].reset_index(drop=True)

sf_data_value_counts = sf_data['example_very_unclear'].value_counts()
# check if it worked
assert sf_data_value_counts[False] == sf_data.shape[0]

from datasets import Dataset

In [ ]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(sf_data)
ds_dict = DatasetDict()
ds_dict['train'] = ds
ds_dict.save_to_disk(DATA_PATH + 'dataset')